Abiy

In [1]:
#!pip install python-dotenv decorator==4.3 networkx
#!pip install pycoingecko
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import pprint
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

%matplotlib inline

In [2]:
bond_df = pd.read_json('./Resources/bond_data.json')
bond_df

,"('GOVT', 'open')","('GOVT', 'high')","('GOVT', 'low')","('GOVT', 'close')","('GOVT', 'volume')","('IGSB', 'open')","('IGSB', 'high')","('IGSB', 'low')","('IGSB', 'close')","('IGSB', 'volume')",...,"('VMBS', 'open')","('VMBS', 'high')","('VMBS', 'low')","('VMBS', 'close')","('VMBS', 'volume')","('VOOG', 'open')","('VOOG', 'high')","('VOOG', 'low')","('VOOG', 'close')","('VOOG', 'volume')"
2017-10-20 04:00:00,25.130,25.170,25.13,25.150,651942,52.605,52.614,52.595,52.605,545736,...,52.65,52.6892,52.6301,52.640,286452,132.00,132.0800,131.7100,131.95,32071
2017-10-23 04:00:00,25.150,25.180,25.15,25.170,472528,52.625,52.640,52.601,52.635,856312,...,52.68,52.7200,52.6700,52.720,262840,132.30,132.3000,131.2500,131.33,29681
2017-10-24 04:00:00,25.110,25.135,25.10,25.120,636516,52.580,52.625,52.580,52.610,1359706,...,52.67,52.6900,52.6100,52.620,326018,131.59,131.5900,131.2099,131.41,39516
2017-10-25 04:00:00,25.070,25.100,25.05,25.090,575142,52.600,52.605,52.570,52.595,413946,...,52.56,52.8600,52.5300,52.570,404450,131.26,131.4175,130.2600,130.94,35633
2017-10-26 04:00:00,25.090,25.110,25.05,25.060,758379,52.585,52.605,52.570,52.588,534592,...,52.64,52.6411,52.5200,52.520,255541,131.04,131.2200,130.8400,131.01,27887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-06 04:00:00,26.490,26.520,26.48,26.500,3504197,54.540,54.560,54.530,54.560,1563571,...,53.25,53.2721,53.2200,53.240,844198,265.05,269.2500,264.2500,269.23,74461
2021-10-07 04:00:00,26.440,26.450,26.40,26.410,3467051,54.550,54.560,54.520,54.530,1897730,...,53.25,53.2600,53.1700,53.175,907122,271.65,273.5000,271.3700,271.58,62066
2021-10-08 04:00:00,26.390,26.390,26.34,26.360,5450632,54.520,54.530,54.470,54.470,975088,...,53.16,53.2000,53.0900,53.090,955001,272.57,272.5700,270.3100,270.50,47044
2021-10-11 04:00:00,26.320,26.345,26.31,26.310,2092922,54.440,54.450,54.410,54.410,620056,...,53.07,53.0900,53.0300,53.030,726022,269.84,272.2100,269.0000,269.00,61897


In [3]:
stock_df = pd.read_json('./Resources/stock_data.json')
stock_df

,"('AAPL', 'open')","('AAPL', 'high')","('AAPL', 'low')","('AAPL', 'close')","('AAPL', 'volume')","('AMZN', 'open')","('AMZN', 'high')","('AMZN', 'low')","('AMZN', 'close')","('AMZN', 'volume')",...,"('AAPL', 'Daily_Returns')","('AMZN', 'Daily_Returns')","('BA', 'Daily_Returns')","('NVDA', 'Daily_Returns')","('BRK.A', 'Daily_Returns')","('GS', 'Daily_Returns')","('COP', 'Daily_Returns')","('GILD', 'Daily_Returns')","('CCOI', 'Daily_Returns')","('PG', 'Daily_Returns')"
2017-10-23 04:00:00,156.89,157.6900,155.5000,156.20,16418890,986.730,986.7750,962.50,966.34,2341953,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-10-24 04:00:00,156.29,157.4200,156.2000,157.09,14016780,969.000,979.8500,965.00,976.00,1744834,...,0.005698,0.009996,0.014103,0.010426,0.001027,0.011191,0.014422,-0.000998,0.012170,-0.003322
2017-10-25 04:00:00,156.91,157.5500,155.2700,156.41,16016689,978.000,984.4400,966.24,972.91,2001000,...,-0.004329,-0.003166,-0.028679,-0.025318,-0.004534,-0.012905,-0.020296,-0.002372,-0.011423,-0.001954
2017-10-26 04:00:00,157.23,157.8295,156.7800,157.42,12374254,980.330,982.9000,968.55,972.54,2373669,...,0.006457,-0.000380,0.003290,0.010070,-0.000159,0.000041,0.029824,-0.025782,0.042976,0.007601
2017-10-27 04:00:00,159.29,163.6000,158.7000,163.03,36735155,1057.030,1105.5800,1050.55,1100.99,11137045,...,0.035637,0.132077,-0.011417,0.031494,-0.001271,0.000207,-0.004665,-0.010149,0.025267,-0.005258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-06 04:00:00,139.58,142.1500,138.3700,141.99,73830085,3213.530,3264.3400,3198.62,3263.20,2186146,...,0.006308,0.013105,0.002251,0.012225,0.007760,0.005831,0.001115,0.001781,0.004143,0.013061
2021-10-07 04:00:00,143.00,144.2150,142.7200,143.28,54652460,3289.530,3325.7460,3283.06,3302.43,2131959,...,0.009085,0.012022,0.006977,0.018116,0.008262,0.006931,-0.003899,0.009781,0.008527,0.008146
2021-10-08 04:00:00,144.04,144.1781,142.5600,143.01,48459122,3316.080,3321.4300,3288.20,3290.57,1708132,...,-0.001884,-0.003591,-0.000574,-0.011815,0.004952,0.005374,0.047393,-0.001174,-0.015001,-0.004146
2021-10-11 04:00:00,142.30,144.8100,141.8100,142.80,55706039,3272.410,3292.5898,3238.10,3245.15,1825823,...,-0.001468,-0.013803,-0.000177,-0.006578,-0.009806,-0.019547,-0.012680,0.000147,-0.002077,0.004092


In [4]:
crypto_df = pd.read_json('./Resources/crypto_data.json')
crypto_df

,"('BTC', 'price')","('BTC', 'market_cap')","('BTC', 'total_volume')","('ETH', 'price')","('ETH', 'market_cap')","('ETH', 'total_volume')","('LTE', 'price')","('LTE', 'market_cap')","('LTE', 'total_volume')","('LINK', 'price')","('LINK', 'market_cap')","('LINK', 'total_volume')","('ADA', 'price')","('ADA', 'market_cap')","('ADA', 'total_volume')"
2017-11-09,7186.292775,1.198168e+11,2.330694e+09,322.329520,3.082374e+10,6.918154e+08,64.537577,3.469079e+09,2.345449e+08,0.225377,7.888183e+07,3.190026e+06,0.032769,8.496170e+08,2.062814e+07
2017-11-10,6656.839694,1.109994e+11,3.886426e+09,301.354933,2.882416e+10,6.505058e+08,59.371318,3.192341e+09,2.366416e+08,0.189036,6.616275e+07,2.121175e+06,0.027293,7.076286e+08,6.563684e+06
2017-11-11,6365.397852,1.061461e+11,3.148370e+09,316.258342,3.025603e+10,6.186761e+08,62.612170,3.367560e+09,2.256785e+08,0.181223,6.342793e+07,2.229298e+06,0.027499,7.129560e+08,5.632138e+06
2017-11-12,5866.513129,9.783217e+10,6.001018e+09,314.166128,3.006218e+10,1.172944e+09,58.916112,3.169837e+09,2.526936e+08,0.170013,5.950462e+07,5.361146e+06,0.024522,6.357746e+08,7.550745e+06
2017-11-13,6423.091402,1.071254e+11,4.636292e+09,315.343956,3.018143e+10,7.833661e+08,61.163042,3.291569e+09,1.543181e+08,0.180691,6.324186e+07,2.942234e+06,0.025114,6.511399e+08,4.284486e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-07,55415.005742,1.044675e+12,4.846239e+10,3592.176130,4.235600e+11,2.470936e+10,179.199934,1.230975e+10,3.225630e+09,26.894355,1.227483e+10,1.278611e+09,2.213307,7.096721e+10,2.144512e+09
2021-10-08,53894.595995,1.015247e+12,3.654982e+10,3594.918358,4.236103e+11,2.118680e+10,179.056333,1.230113e+10,3.515152e+09,26.890238,1.224418e+10,1.174577e+09,2.279964,7.298744e+10,2.669937e+09
2021-10-09,54010.937474,1.016651e+12,3.393549e+10,3558.549577,4.199184e+11,1.814447e+10,175.969071,1.209022e+10,3.002931e+09,26.266825,1.204293e+10,8.959017e+08,2.230678,7.158880e+10,1.521105e+09
2021-10-10,55125.460338,1.038525e+12,3.192734e+10,3588.080922,4.218362e+11,1.381563e+10,179.983616,1.236738e+10,3.043831e+09,27.445360,1.256066e+10,9.103347e+08,2.269474,7.276435e+10,1.322555e+09


In [5]:
#zero_risk=bond_df[["TLT","VGLT","VOOG","VMBS","IWF","IGSB","GOVT","AGZ"]]

In [6]:
new_df = pd.concat([bond_df,stock_df,crypto_df], axis=1)
new_df

ticker_names =["TLT","VGLT","VOOG","VMBS","IWF","IGSB","GOVT","AGZ"]
# new_df.rename(columns={"('GOVT', 'close')":"GOVT_close", "('GOVT', 'low')":"GOVT_low"})
columns = new_df.columns
columns
# ticker_columns = [x for x in columns if x in ticker_names]
ticker_columns = []
for column in columns:
    for ticker_name in ticker_names:
        if ticker_name in column:
            ticker_columns.append(column)
    

ticker_columns
new_df[ticker_columns].dropna()

,"('GOVT', 'open')","('GOVT', 'high')","('GOVT', 'low')","('GOVT', 'close')","('GOVT', 'volume')","('IGSB', 'open')","('IGSB', 'high')","('IGSB', 'low')","('IGSB', 'close')","('IGSB', 'volume')",...,"('VMBS', 'open')","('VMBS', 'high')","('VMBS', 'low')","('VMBS', 'close')","('VMBS', 'volume')","('VOOG', 'open')","('VOOG', 'high')","('VOOG', 'low')","('VOOG', 'close')","('VOOG', 'volume')"
2017-10-20 04:00:00,25.130,25.170,25.13,25.150,651942.0,52.605,52.614,52.595,52.605,545736.0,...,52.65,52.6892,52.6301,52.640,286452.0,132.00,132.0800,131.7100,131.95,32071.0
2017-10-23 04:00:00,25.150,25.180,25.15,25.170,472528.0,52.625,52.640,52.601,52.635,856312.0,...,52.68,52.7200,52.6700,52.720,262840.0,132.30,132.3000,131.2500,131.33,29681.0
2017-10-24 04:00:00,25.110,25.135,25.10,25.120,636516.0,52.580,52.625,52.580,52.610,1359706.0,...,52.67,52.6900,52.6100,52.620,326018.0,131.59,131.5900,131.2099,131.41,39516.0
2017-10-25 04:00:00,25.070,25.100,25.05,25.090,575142.0,52.600,52.605,52.570,52.595,413946.0,...,52.56,52.8600,52.5300,52.570,404450.0,131.26,131.4175,130.2600,130.94,35633.0
2017-10-26 04:00:00,25.090,25.110,25.05,25.060,758379.0,52.585,52.605,52.570,52.588,534592.0,...,52.64,52.6411,52.5200,52.520,255541.0,131.04,131.2200,130.8400,131.01,27887.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-06 04:00:00,26.490,26.520,26.48,26.500,3504197.0,54.540,54.560,54.530,54.560,1563571.0,...,53.25,53.2721,53.2200,53.240,844198.0,265.05,269.2500,264.2500,269.23,74461.0
2021-10-07 04:00:00,26.440,26.450,26.40,26.410,3467051.0,54.550,54.560,54.520,54.530,1897730.0,...,53.25,53.2600,53.1700,53.175,907122.0,271.65,273.5000,271.3700,271.58,62066.0
2021-10-08 04:00:00,26.390,26.390,26.34,26.360,5450632.0,54.520,54.530,54.470,54.470,975088.0,...,53.16,53.2000,53.0900,53.090,955001.0,272.57,272.5700,270.3100,270.50,47044.0
2021-10-11 04:00:00,26.320,26.345,26.31,26.310,2092922.0,54.440,54.450,54.410,54.410,620056.0,...,53.07,53.0900,53.0300,53.030,726022.0,269.84,272.2100,269.0000,269.00,61897.0


In [7]:
# import yfinance as yf 
# data = yf.download(["TLT","VGLT","VOOG","VMBS","IWF","IGSB","GOVT","AGZ","AAPL","AMZN","CCOI", "PG","BA","NVDA","BRK-A","GS","COP","GILD","BTC","ETH","LTE.V","ADA-USD" ],'2017-01-01','2021-10-04')
# data.dropna()

In [8]:
#!pip install pandas_datareader ,"IWF","IGSB","GOVT","AGZ"

In [9]:
#Based on Risk level
#Investments That Provide Various Levels Of Risk & Return

In [10]:
#Zero risk portfolio: Treasury bonds held to maturity, money market accounts, and CDs where the FDIC guarantees up to $250,000 in losses per person.
#  Treasury bonds(100% )
zero_risk_portfolio=new_df[ticker_columns].dropna()
zero_risk_portfolio

,"('GOVT', 'open')","('GOVT', 'high')","('GOVT', 'low')","('GOVT', 'close')","('GOVT', 'volume')","('IGSB', 'open')","('IGSB', 'high')","('IGSB', 'low')","('IGSB', 'close')","('IGSB', 'volume')",...,"('VMBS', 'open')","('VMBS', 'high')","('VMBS', 'low')","('VMBS', 'close')","('VMBS', 'volume')","('VOOG', 'open')","('VOOG', 'high')","('VOOG', 'low')","('VOOG', 'close')","('VOOG', 'volume')"
2017-10-20 04:00:00,25.130,25.170,25.13,25.150,651942.0,52.605,52.614,52.595,52.605,545736.0,...,52.65,52.6892,52.6301,52.640,286452.0,132.00,132.0800,131.7100,131.95,32071.0
2017-10-23 04:00:00,25.150,25.180,25.15,25.170,472528.0,52.625,52.640,52.601,52.635,856312.0,...,52.68,52.7200,52.6700,52.720,262840.0,132.30,132.3000,131.2500,131.33,29681.0
2017-10-24 04:00:00,25.110,25.135,25.10,25.120,636516.0,52.580,52.625,52.580,52.610,1359706.0,...,52.67,52.6900,52.6100,52.620,326018.0,131.59,131.5900,131.2099,131.41,39516.0
2017-10-25 04:00:00,25.070,25.100,25.05,25.090,575142.0,52.600,52.605,52.570,52.595,413946.0,...,52.56,52.8600,52.5300,52.570,404450.0,131.26,131.4175,130.2600,130.94,35633.0
2017-10-26 04:00:00,25.090,25.110,25.05,25.060,758379.0,52.585,52.605,52.570,52.588,534592.0,...,52.64,52.6411,52.5200,52.520,255541.0,131.04,131.2200,130.8400,131.01,27887.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-06 04:00:00,26.490,26.520,26.48,26.500,3504197.0,54.540,54.560,54.530,54.560,1563571.0,...,53.25,53.2721,53.2200,53.240,844198.0,265.05,269.2500,264.2500,269.23,74461.0
2021-10-07 04:00:00,26.440,26.450,26.40,26.410,3467051.0,54.550,54.560,54.520,54.530,1897730.0,...,53.25,53.2600,53.1700,53.175,907122.0,271.65,273.5000,271.3700,271.58,62066.0
2021-10-08 04:00:00,26.390,26.390,26.34,26.360,5450632.0,54.520,54.530,54.470,54.470,975088.0,...,53.16,53.2000,53.0900,53.090,955001.0,272.57,272.5700,270.3100,270.50,47044.0
2021-10-11 04:00:00,26.320,26.345,26.31,26.310,2092922.0,54.440,54.450,54.410,54.410,620056.0,...,53.07,53.0900,53.0300,53.030,726022.0,269.84,272.2100,269.0000,269.00,61897.0


In [11]:
zero_risk_portfolio.to_json('zero_risk_portfolio.json')

In [12]:
#Lower Risk portfolio (80% bonds and 20% stocks)

In [13]:
tickers_names =["TLT","VGLT","VOOG","VMBS","IWF","IGSB","GOVT","AGZ","AAPL","AMZN"]
# new_df.rename(columns={"('GOVT', 'close')":"GOVT_close", "('GOVT', 'low')":"GOVT_low"})
columns = new_df.columns
columns
# ticker_columns = [x for x in columns if x in ticker_names]
tickers_columns = []
for column in columns:
    for ticker_name in tickers_names:
        if ticker_name in column:
            tickers_columns.append(column)
    

# ticker_columns
# new_df[tickers_columns]

In [14]:
zero_risk_portfolio.to_json('zero_risk_portfolio.json')

In [15]:
lower_risk_portfolio =new_df[tickers_columns].dropna()
lower_risk_portfolio

,"('GOVT', 'open')","('GOVT', 'high')","('GOVT', 'low')","('GOVT', 'close')","('GOVT', 'volume')","('IGSB', 'open')","('IGSB', 'high')","('IGSB', 'low')","('IGSB', 'close')","('IGSB', 'volume')",...,"('AAPL', 'low')","('AAPL', 'close')","('AAPL', 'volume')","('AMZN', 'open')","('AMZN', 'high')","('AMZN', 'low')","('AMZN', 'close')","('AMZN', 'volume')","('AAPL', 'Daily_Returns')","('AMZN', 'Daily_Returns')"
2017-10-24 04:00:00,25.110,25.135,25.10,25.120,636516.0,52.580,52.625,52.580,52.610,1359706.0,...,156.2000,157.09,14016780.0,969.000,979.8500,965.00,976.00,1744834.0,0.005698,0.009996
2017-10-25 04:00:00,25.070,25.100,25.05,25.090,575142.0,52.600,52.605,52.570,52.595,413946.0,...,155.2700,156.41,16016689.0,978.000,984.4400,966.24,972.91,2001000.0,-0.004329,-0.003166
2017-10-26 04:00:00,25.090,25.110,25.05,25.060,758379.0,52.585,52.605,52.570,52.588,534592.0,...,156.7800,157.42,12374254.0,980.330,982.9000,968.55,972.54,2373669.0,0.006457,-0.000380
2017-10-27 04:00:00,25.070,25.120,25.07,25.120,813036.0,52.565,52.630,52.565,52.605,446882.0,...,158.7000,163.03,36735155.0,1057.030,1105.5800,1050.55,1100.99,11137045.0,0.035637,0.132077
2017-10-30 04:00:00,25.160,25.190,25.13,25.180,530322.0,52.635,52.640,52.605,52.613,789298.0,...,163.7200,166.72,37107049.0,1095.420,1122.7900,1093.56,1110.85,4459741.0,0.022634,0.008956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-06 04:00:00,26.490,26.520,26.48,26.500,3504197.0,54.540,54.560,54.530,54.560,1563571.0,...,138.3700,141.99,73830085.0,3213.530,3264.3400,3198.62,3263.20,2186146.0,0.006308,0.013105
2021-10-07 04:00:00,26.440,26.450,26.40,26.410,3467051.0,54.550,54.560,54.520,54.530,1897730.0,...,142.7200,143.28,54652460.0,3289.530,3325.7460,3283.06,3302.43,2131959.0,0.009085,0.012022
2021-10-08 04:00:00,26.390,26.390,26.34,26.360,5450632.0,54.520,54.530,54.470,54.470,975088.0,...,142.5600,143.01,48459122.0,3316.080,3321.4300,3288.20,3290.57,1708132.0,-0.001884,-0.003591
2021-10-11 04:00:00,26.320,26.345,26.31,26.310,2092922.0,54.440,54.450,54.410,54.410,620056.0,...,141.8100,142.80,55706039.0,3272.410,3292.5898,3238.10,3245.15,1825823.0,-0.001468,-0.013803


In [16]:
lower_risk_portfolio.to_json('lower_risk_portfolio.json')

In [17]:
tickers_names1 =["TLT","VGLT","VOOG","VMBS","AAPL","AMZN","BA","NVDA","BRK-A","GS"]
# new_df.rename(columns={"('GOVT', 'close')":"GOVT_close", "('GOVT', 'low')":"GOVT_low"})
columns = new_df.columns
columns
# ticker_columns = [x for x in columns if x in ticker_names]
tickers_columns1 = []
for column in columns:
    for ticker_name in tickers_names1:
        if ticker_name in column:
            tickers_columns1.append(column)
    

# ticker_columns
# new_df[tickers_columns1]

In [18]:
#Low Medium Risk portfolio(70% bonds and 30% stocks)
low_medium_risk_portfolio=new_df[tickers_columns1].dropna()
low_medium_risk_portfolio

,"('IGSB', 'open')","('IGSB', 'high')","('IGSB', 'low')","('IGSB', 'close')","('IGSB', 'volume')","('TLT', 'open')","('TLT', 'high')","('TLT', 'low')","('TLT', 'close')","('TLT', 'volume')",...,"('GS', 'open')","('GS', 'high')","('GS', 'low')","('GS', 'close')","('GS', 'volume')","('AAPL', 'Daily_Returns')","('AMZN', 'Daily_Returns')","('BA', 'Daily_Returns')","('NVDA', 'Daily_Returns')","('GS', 'Daily_Returns')"
2017-10-24 04:00:00,52.580,52.625,52.580,52.610,1359706.0,123.370,123.690,123.150,123.45,5965628.0,...,243.60,246.25,243.00,244.87,1503543.0,0.005698,0.009996,0.014103,0.010426,0.011191
2017-10-25 04:00:00,52.600,52.605,52.570,52.595,413946.0,122.530,122.980,122.420,122.85,7943533.0,...,246.16,246.50,240.80,241.71,2072140.0,-0.004329,-0.003166,-0.028679,-0.025318,-0.012905
2017-10-26 04:00:00,52.585,52.605,52.570,52.588,534592.0,123.060,123.060,122.430,122.45,7061633.0,...,242.33,244.00,241.53,241.72,1430800.0,0.006457,-0.000380,0.003290,0.010070,0.000041
2017-10-27 04:00:00,52.565,52.630,52.565,52.605,446882.0,122.940,123.325,122.750,123.22,7868465.0,...,240.85,242.44,239.67,241.77,1354379.0,0.035637,0.132077,-0.011417,0.031494,0.000207
2017-10-30 04:00:00,52.635,52.640,52.605,52.613,789298.0,123.870,124.460,123.560,124.44,6502154.0,...,240.10,242.59,239.53,240.86,1324530.0,0.022634,0.008956,0.011431,0.009963,-0.003764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-06 04:00:00,54.540,54.560,54.530,54.560,1563571.0,144.230,144.715,144.025,144.39,16697508.0,...,382.38,389.55,378.66,388.10,1860435.0,0.006308,0.013105,0.002251,0.012225,0.005831
2021-10-07 04:00:00,54.550,54.560,54.520,54.530,1897730.0,143.185,143.335,142.590,142.87,12978486.0,...,393.71,397.85,389.76,390.79,1955131.0,0.009085,0.012022,0.006977,0.018116,0.006931
2021-10-08 04:00:00,54.520,54.530,54.470,54.470,975088.0,142.255,142.350,141.525,141.90,15033306.0,...,390.83,394.31,388.18,392.89,1273715.0,-0.001884,-0.003591,-0.000574,-0.011815,0.005374
2021-10-11 04:00:00,54.440,54.450,54.410,54.410,620056.0,141.600,141.900,141.450,141.50,6461811.0,...,392.90,396.95,385.02,385.21,1616537.0,-0.001468,-0.013803,-0.000177,-0.006578,-0.019547


In [19]:
low_medium_risk_portfolio.to_json('low_medium_risk_portfolio.json')

In [20]:
tickers_names2 =["TLT","VGLT","VOOG","VMBS","AAPL","AMZN","BA","NVDA","BRK-A","GS"]
# new_df.rename(columns={"('GOVT', 'close')":"GOVT_close", "('GOVT', 'low')":"GOVT_low"})
columns = new_df.columns
columns
# ticker_columns = [x for x in columns if x in ticker_names]
tickers_columns2 = []
for column in columns:
    for ticker_name in tickers_names2:
        if ticker_name in column:
            tickers_columns2.append(column)
    
# new_df[tickers_columns2]

In [21]:
#Medium Risk Portfolio (60% stocks and 40% bonds)

In [22]:
medium_risk_portfolio= new_df[tickers_columns2].dropna()
medium_risk_portfolio

,"('IGSB', 'open')","('IGSB', 'high')","('IGSB', 'low')","('IGSB', 'close')","('IGSB', 'volume')","('TLT', 'open')","('TLT', 'high')","('TLT', 'low')","('TLT', 'close')","('TLT', 'volume')",...,"('GS', 'open')","('GS', 'high')","('GS', 'low')","('GS', 'close')","('GS', 'volume')","('AAPL', 'Daily_Returns')","('AMZN', 'Daily_Returns')","('BA', 'Daily_Returns')","('NVDA', 'Daily_Returns')","('GS', 'Daily_Returns')"
2017-10-24 04:00:00,52.580,52.625,52.580,52.610,1359706.0,123.370,123.690,123.150,123.45,5965628.0,...,243.60,246.25,243.00,244.87,1503543.0,0.005698,0.009996,0.014103,0.010426,0.011191
2017-10-25 04:00:00,52.600,52.605,52.570,52.595,413946.0,122.530,122.980,122.420,122.85,7943533.0,...,246.16,246.50,240.80,241.71,2072140.0,-0.004329,-0.003166,-0.028679,-0.025318,-0.012905
2017-10-26 04:00:00,52.585,52.605,52.570,52.588,534592.0,123.060,123.060,122.430,122.45,7061633.0,...,242.33,244.00,241.53,241.72,1430800.0,0.006457,-0.000380,0.003290,0.010070,0.000041
2017-10-27 04:00:00,52.565,52.630,52.565,52.605,446882.0,122.940,123.325,122.750,123.22,7868465.0,...,240.85,242.44,239.67,241.77,1354379.0,0.035637,0.132077,-0.011417,0.031494,0.000207
2017-10-30 04:00:00,52.635,52.640,52.605,52.613,789298.0,123.870,124.460,123.560,124.44,6502154.0,...,240.10,242.59,239.53,240.86,1324530.0,0.022634,0.008956,0.011431,0.009963,-0.003764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-06 04:00:00,54.540,54.560,54.530,54.560,1563571.0,144.230,144.715,144.025,144.39,16697508.0,...,382.38,389.55,378.66,388.10,1860435.0,0.006308,0.013105,0.002251,0.012225,0.005831
2021-10-07 04:00:00,54.550,54.560,54.520,54.530,1897730.0,143.185,143.335,142.590,142.87,12978486.0,...,393.71,397.85,389.76,390.79,1955131.0,0.009085,0.012022,0.006977,0.018116,0.006931
2021-10-08 04:00:00,54.520,54.530,54.470,54.470,975088.0,142.255,142.350,141.525,141.90,15033306.0,...,390.83,394.31,388.18,392.89,1273715.0,-0.001884,-0.003591,-0.000574,-0.011815,0.005374
2021-10-11 04:00:00,54.440,54.450,54.410,54.410,620056.0,141.600,141.900,141.450,141.50,6461811.0,...,392.90,396.95,385.02,385.21,1616537.0,-0.001468,-0.013803,-0.000177,-0.006578,-0.019547


In [23]:
medium_risk_portfolio.to_json('crypto_data.json')

In [24]:
tickers_names3 =["TLT","VGLT","VOOG","VMBS","AAPL","AMZN","BA","NVDA","BRK-A","GS"]
# new_df.rename(columns={"('GOVT', 'close')":"GOVT_close", "('GOVT', 'low')":"GOVT_low"})
columns = new_df.columns
columns
# ticker_columns = [x for x in columns if x in ticker_names]
tickers_columns3 = []
for column in columns:
    for ticker_name3 in tickers_names3:
        if ticker_name in column:
            tickers_columns3.append(column)
# new_df[tickers_columns3]

In [25]:
#Medium High Risk portfolio(70% stocks and 30% bonds)
medium_high_risk_portfolio = new_df[tickers_columns3].dropna()
medium_high_risk_portfolio

,"('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')",...,"('GS', 'Daily_Returns')","('GS', 'Daily_Returns')","('GS', 'Daily_Returns')","('GS', 'Daily_Returns')","('GS', 'Daily_Returns')","('GS', 'Daily_Returns')","('GS', 'Daily_Returns')","('GS', 'Daily_Returns')","('GS', 'Daily_Returns')","('GS', 'Daily_Returns')"
2017-10-24 04:00:00,52.580,52.580,52.580,52.580,52.580,52.580,52.580,52.580,52.580,52.580,...,0.011191,0.011191,0.011191,0.011191,0.011191,0.011191,0.011191,0.011191,0.011191,0.011191
2017-10-25 04:00:00,52.600,52.600,52.600,52.600,52.600,52.600,52.600,52.600,52.600,52.600,...,-0.012905,-0.012905,-0.012905,-0.012905,-0.012905,-0.012905,-0.012905,-0.012905,-0.012905,-0.012905
2017-10-26 04:00:00,52.585,52.585,52.585,52.585,52.585,52.585,52.585,52.585,52.585,52.585,...,0.000041,0.000041,0.000041,0.000041,0.000041,0.000041,0.000041,0.000041,0.000041,0.000041
2017-10-27 04:00:00,52.565,52.565,52.565,52.565,52.565,52.565,52.565,52.565,52.565,52.565,...,0.000207,0.000207,0.000207,0.000207,0.000207,0.000207,0.000207,0.000207,0.000207,0.000207
2017-10-30 04:00:00,52.635,52.635,52.635,52.635,52.635,52.635,52.635,52.635,52.635,52.635,...,-0.003764,-0.003764,-0.003764,-0.003764,-0.003764,-0.003764,-0.003764,-0.003764,-0.003764,-0.003764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-06 04:00:00,54.540,54.540,54.540,54.540,54.540,54.540,54.540,54.540,54.540,54.540,...,0.005831,0.005831,0.005831,0.005831,0.005831,0.005831,0.005831,0.005831,0.005831,0.005831
2021-10-07 04:00:00,54.550,54.550,54.550,54.550,54.550,54.550,54.550,54.550,54.550,54.550,...,0.006931,0.006931,0.006931,0.006931,0.006931,0.006931,0.006931,0.006931,0.006931,0.006931
2021-10-08 04:00:00,54.520,54.520,54.520,54.520,54.520,54.520,54.520,54.520,54.520,54.520,...,0.005374,0.005374,0.005374,0.005374,0.005374,0.005374,0.005374,0.005374,0.005374,0.005374
2021-10-11 04:00:00,54.440,54.440,54.440,54.440,54.440,54.440,54.440,54.440,54.440,54.440,...,-0.019547,-0.019547,-0.019547,-0.019547,-0.019547,-0.019547,-0.019547,-0.019547,-0.019547,-0.019547


In [26]:
medium_high_risk_portfolio.to_json('medium_high_risk_portfolio.json',orient= "split")

In [27]:
#High Risk portfolio(80% stocks and 20% cryptocurrency)

In [28]:
tickers_names4 =["AAPL","AMZN", "BA","NVDA","BRK-A","GS","COP","GILD","BTC","ETH"]
# new_df.rename(columns={"('GOVT', 'close')":"GOVT_close", "('GOVT', 'low')":"GOVT_low"})
columns = new_df.columns
columns
# ticker_columns = [x for x in columns if x in ticker_names]
tickers_columns4 = []
for column in columns:
    for ticker_name4 in tickers_names4:
        if ticker_name in column:
            tickers_columns4.append(column)
high_risk_portfolio=  new_df[tickers_columns4].dropna()
high_risk_portfolio

,"('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')",...,"('GS', 'Daily_Returns')","('GS', 'Daily_Returns')","('GS', 'Daily_Returns')","('GS', 'Daily_Returns')","('GS', 'Daily_Returns')","('GS', 'Daily_Returns')","('GS', 'Daily_Returns')","('GS', 'Daily_Returns')","('GS', 'Daily_Returns')","('GS', 'Daily_Returns')"
2017-10-24 04:00:00,52.580,52.580,52.580,52.580,52.580,52.580,52.580,52.580,52.580,52.580,...,0.011191,0.011191,0.011191,0.011191,0.011191,0.011191,0.011191,0.011191,0.011191,0.011191
2017-10-25 04:00:00,52.600,52.600,52.600,52.600,52.600,52.600,52.600,52.600,52.600,52.600,...,-0.012905,-0.012905,-0.012905,-0.012905,-0.012905,-0.012905,-0.012905,-0.012905,-0.012905,-0.012905
2017-10-26 04:00:00,52.585,52.585,52.585,52.585,52.585,52.585,52.585,52.585,52.585,52.585,...,0.000041,0.000041,0.000041,0.000041,0.000041,0.000041,0.000041,0.000041,0.000041,0.000041
2017-10-27 04:00:00,52.565,52.565,52.565,52.565,52.565,52.565,52.565,52.565,52.565,52.565,...,0.000207,0.000207,0.000207,0.000207,0.000207,0.000207,0.000207,0.000207,0.000207,0.000207
2017-10-30 04:00:00,52.635,52.635,52.635,52.635,52.635,52.635,52.635,52.635,52.635,52.635,...,-0.003764,-0.003764,-0.003764,-0.003764,-0.003764,-0.003764,-0.003764,-0.003764,-0.003764,-0.003764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-06 04:00:00,54.540,54.540,54.540,54.540,54.540,54.540,54.540,54.540,54.540,54.540,...,0.005831,0.005831,0.005831,0.005831,0.005831,0.005831,0.005831,0.005831,0.005831,0.005831
2021-10-07 04:00:00,54.550,54.550,54.550,54.550,54.550,54.550,54.550,54.550,54.550,54.550,...,0.006931,0.006931,0.006931,0.006931,0.006931,0.006931,0.006931,0.006931,0.006931,0.006931
2021-10-08 04:00:00,54.520,54.520,54.520,54.520,54.520,54.520,54.520,54.520,54.520,54.520,...,0.005374,0.005374,0.005374,0.005374,0.005374,0.005374,0.005374,0.005374,0.005374,0.005374
2021-10-11 04:00:00,54.440,54.440,54.440,54.440,54.440,54.440,54.440,54.440,54.440,54.440,...,-0.019547,-0.019547,-0.019547,-0.019547,-0.019547,-0.019547,-0.019547,-0.019547,-0.019547,-0.019547


In [35]:
high_risk_portfolio.to_json('high_risk_portfolio.json', orient='split')